In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit import transpile

In [2]:
import numpy as np

In [3]:
n = 50
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.randint(2, size=n)

In [4]:
alice_bits

array([1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0])

In [5]:
alice_bases

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0])

In [6]:
circuits = []
for i in range(n):
    qc = QuantumCircuit(1, 1)
    if alice_bits[i] == 1:
        qc.x(0)  # encode bit 1
    if alice_bases[i] == 1:
        qc.h(0)  # change to X basis
    circuits.append(qc)

In [12]:
bob_bases = np.random.randint(2, size=n)
results = []
sim = Aer.get_backend('aer_simulator')

In [13]:
for i in range(n):
    qc = circuits[i].copy()
    if bob_bases[i] == 1:
        qc.h(0)
    qc.measure(0, 0)
    qc = transpile(qc, sim)
    result = sim.run(qc, shots=1, memory=True).result()
    bit = int(result.get_memory()[0])
    results.append(bit)

In [14]:
# Step 4: Compare bases and extract key
same_bases = [i for i in range(n) if alice_bases[i] == bob_bases[i]]
key_alice = [alice_bits[i] for i in same_bases]
key_bob = [results[i] for i in same_bases]

In [18]:
print("Alice bits: ", alice_bits)
print("Alice bases:", alice_bases)
print("Bob bases:  ", bob_bases)
print("Bob results:", results)
print("\nIndices with same bases:", same_bases)
print("Alice key:", key_alice)
print("Bob key:  ", key_bob)

Alice bits:  [1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0
 1 1 0 1 1 1 0 1 1 1 0 1 0]
Alice bases: [1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1
 0 1 1 0 1 0 1 1 1 1 0 0 0]
Bob bases:   [0 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1
 0 1 1 1 0 0 0 0 0 1 1 0 1]
Bob results: [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]

Indices with same bases: [1, 2, 3, 4, 6, 7, 10, 17, 18, 19, 21, 22, 26, 27, 30, 32, 33, 35, 36, 37, 38, 39, 42, 46, 48]
Alice key: [np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1)]
Bob key:   [1, 1, 0, 1, 1, 1, 

In [19]:
errors = sum(a != b for a, b in zip(key_alice, key_bob))
qber = errors / len(key_alice) if key_alice else 0

In [20]:
print(f"\nQuantum Bit Error Rate (QBER): {qber*100:.1f}%")


Quantum Bit Error Rate (QBER): 0.0%
